In [1]:
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt

C:\Users\Kamila Nita\AppData\Roaming\Python\Python39\site-packages\pyspark\pandas\__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
spark = SparkSession.builder.appName('IO').getOrCreate()

In [12]:
summer_athletes = spark.read.csv("Athletes_summer_games.csv", 
                      header='true', 
                      inferSchema='true')

summer_athletes = summer_athletes.withColumn("index_col", summer_athletes["_c0"]).drop("_c0")

In [13]:
summer_athletes.show()

+--------------------+---+----+--------------+---+-----------+----+------+-----------+----------+--------------------+------+---------+
|                Name|Sex| Age|          Team|NOC|      Games|Year|Season|       City|     Sport|               Event| Medal|index_col|
+--------------------+---+----+--------------+---+-----------+----+------+-----------+----------+--------------------+------+---------+
|           A Dijiang|  M|24.0|         China|CHN|1992 Summer|1992|Summer|  Barcelona|Basketball|Basketball Men's ...|  null|        0|
|            A Lamusi|  M|23.0|         China|CHN|2012 Summer|2012|Summer|     London|      Judo|Judo Men's Extra-...|  null|        1|
| Gunnar Nielsen Aaby|  M|24.0|       Denmark|DEN|1920 Summer|1920|Summer|  Antwerpen|  Football|Football Men's Fo...|  null|        2|
|Edgar Lindenau Aabye|  M|34.0|Denmark/Sweden|DEN|1900 Summer|1900|Summer|      Paris|Tug-Of-War|Tug-Of-War Men's ...|  Gold|        3|
|"Cornelia ""Cor""...|  F|18.0|   Netherlands|NE

In [14]:
summer_athletes.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Games: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Event: string (nullable = true)
 |-- Medal: string (nullable = true)
 |-- index_col: integer (nullable = true)



In [17]:
num_row = summer_athletes.count()
num_col = len(summer_athletes.columns)

print("Summer Athletes Data Frame shape: ", num_col, ".", num_row)

Summer Athletes Data Frame shape:  13 . 237673


In [20]:
desc = summer_athletes.describe()

desc.show()

+-------+--------------------+-------------+------------------+--------------------+------------------+-----------+------------------+------------------+------------------+--------------+--------------------+--------------------+------------------+
|summary|                Name|          Sex|               Age|                Team|               NOC|      Games|              Year|            Season|              City|         Sport|               Event|               Medal|         index_col|
+-------+--------------------+-------------+------------------+--------------------+------------------+-----------+------------------+------------------+------------------+--------------+--------------------+--------------------+------------------+
|  count|              237673|       237673|            228486|              237671|            237673|     237673|            237673|            237673|            237673|        237673|              237673|               36871|            237673|
|   

In [21]:
count_value_of_teams = summer_athletes.groupBy("Team").agg(count("*").alias("count"))

NameError: name 'count' is not defined